In [1]:
# Importamos las librerías que vamos a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import sklearn_pandas
from matplotlib import gridspec

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
races = pd.read_csv('./csvs/races.csv')
circuits = pd.read_csv('./csvs/circuits.csv')
results = pd.read_csv('./csvs/results.csv')
driver_standings = pd.read_csv('./csvs/driver_standings.csv')
constructor_standings = pd.read_csv('./csvs/constructor_standings.csv')
qualifying = pd.read_csv('./csvs/qualifying.csv')

In [24]:
# df.drop(['column_nameA', 'column_nameB'], axis=1, inplace=True)
races.drop(['year', 'round', 'name', 'url', 'fp1_date', 'fp1_time','fp2_date', 'fp2_time','fp3_date',
            'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis=1, inplace=True)

circuits.drop(['name', 'location', 'country', 'lat', 'lng', 'url'], axis=1, inplace=True)

results.drop(['number', 'position', 'positionText', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 
           'fastestLapTime', 'fastestLapSpeed', 'statusId', 'rank'], axis=1, inplace=True)

driver_standings.drop(['positionText'], axis=1, inplace=True)

constructor_standings.drop(['positionText'], axis=1, inplace=True)

qualifying.drop(['number'], axis=1, inplace=True)

In [26]:
results.head()

,resultId,raceId,driverId,constructorId,grid,positionOrder
0,1,18,1,1,1,1
1,2,18,2,2,5,2
2,3,18,3,3,7,3
3,4,18,4,4,11,4
4,5,18,5,1,3,5


In [ ]:
races.set_index('raceId')
circuits.set_index('circuitId')
results.set_index('raceId')
driver_standings.set_index('raceId')
constructor_standings.set_index('raceId')
qualifying.set_index('raceId')

In [ ]:
ejemplo_join = pd.merge(results, qualifying, how = 'inner', on=['raceId', 'driverId', 'constructorId'], suffixes=['_1','_2'])

In [ ]:
ejemplo_join.head()

In [ ]:
ejemplo_join = ejemplo_join.merge(driver_standings, how='inner', on=['raceId', 'driverId'], suffixes=['_3','_4'])

In [ ]:
ejemplo_join.sample(5)

In [ ]:
ejemplo_join = ejemplo_join.merge(constructor_standings, how='inner', on=['raceId', 'constructorId'], suffixes=['_5','_6'])

In [ ]:
ejemplo_join.head()